# kedro-graphql roadmap

This notebook outlines the feature roadmap for the kedro-graphql project.  The features outlined in this roadmap help to further refine its value proposition.  Kedro-graphql is an opinionated method for deploying kedro projects that fits well into a microservice achitecture.  In addition, it offers or will offer:

  - [FAIR principles](https://www.go-fair.org/fair-principles/) - kedro-graphql's API enables datasets to be Findable, Accessible, Interoperable, and Reusable 
  - an extensible GraphQL API that enables integration with other services
  - an extensible GUI to enable rapid protoyping, accessiblity, and delivering pipelines to non coding end users

Here is a list of features on this roadmap:

- [CRUD pattern](#CRUD-pattern)
- [dataset uploads and downloads](#dataset-uploads-and-downloads)
- [pipeline lineage and version tracking](#pipeline-lineage-and-version-tracking)
- [pipeline staging](#pipeline-staging)
- [pipeline validation](#pipeline-validation)
- [pipeline log handling](#pipeline-log-handling)
- [client module](#client-module)
- [KedroGraphqlRunner module](#KedroGraphqlRunner-module)
- [viz module](#viz-module)

# CRUD pattern
-----

Follow the CRUD (Create, Read, Update, Delete) model and implement the following on the GraphQL API.

- createPipeline (Mutation)
- readPipeline (Query)
- readPipelines (Query)
- updatePipeline (Mutation)
- deletePipeline (Mutation)

# dataset uploads and downloads
-----
Implement dataset uploads and downloads "out-of-band" from kedro-graphql by using the ["signed URL" approach](https://www.apollographql.com/docs/apollo-server/v3/data/file-uploads).  

**implementation**

- At this point in time we only plan to implement the Create and Read actions of the CRUD pattern for the DataSet object

- Modify the DataSet object to have an additional field called "preSignedUrlCreate", the resolver for this field should return a pre-signed url that the client can use to upoad the file directly to the preferred storage backend.  Resolver should be exensible to support different storage backends e.g. traditional filesystem, S3, etc...

```
@strawberry.type
class DataSet:
    name: str
    config: Optional[str] = None
    ...
    preSignedUrlCreate: Optional[str] = None
```

- Modify the DataSet object to have an additional field called "preSignedUrlRead", the resolver for this field should return a pre-signed url that the client can use to read/download the file directly from the preferred storage backend.   Resolver should be exensible to support different storage backends e.g. traditional filesystem, S3, etc...

```
@strawberry.type
class DataSet:
    id: Optional[uuid.UUID] = None
    name: str
    config: Optional[str] = None
    ...
    preSignedUrlRead: Optional[str] = None
```

# pipeline lineage and version tracking
-----
Support tracking pipeline lineage for use cases such as batch workloads (a pipeline that makes calls to the api to run many instances of a pipeline) and one pipeline triggering another.

- update the following objects to have an attribute called "parent" of type "Optional[uuid.UUID] = None" and "version" of type "str".
  - kedro_graphql.models.PipelineInput
  - kedro_graphql.models.Pipeline

```python
from importlib import import_module

@strawberry.type
class Pipeline:
    id: Optional[uuid.UUID] = None
    parent: Optional[uuid.UUID] = None
    version: String
```


# pipeline staging
-----

Currently, the supported states for Pipelines are those in the celery ecosystem:

In [69]:
from celery.states import ALL_STATES
ALL_STATES

frozenset({'STARTED', 'RETRY', 'FAILURE', 'SUCCESS', 'REVOKED', 'PENDING', 'RECEIVED'})

This feature adds the "STAGED" and "READY" as available states.

Expected behavior:

- Allow clients to "stage" a pipeline by specifying "status: STAGED" field on the PipelineInput object when calling createPipeline.  
  - this enables a client to call createPipeline and recieve presigned urls for dataset uploads, upload datasets, then call updatePipeline to change the status from "STAGED" to "READY".
- A call to createPipeline with status = "STAGED" should not run the pipeline
- A call to createPipeline with status = "READY" should run the pipeline
- A call to updatePipeline with status = "READY" should run the pipeline

# pipeline validation
-----

This feature ensures pipelines are "valid" prior to scheduling them for execution by checking to make sure the free inputs exist or have been provided.  There are probably some existing mechanisms already implemented by the kedro project we may be able to reuse for this implementation.

Expected behavior:

- A call to createPipeline with status = "READY" should run the pipeline check to make sure the free inputs exist prior to running
  - if not valid create pipeline with status = "STAGED" and return error response
- A call to updatePipeline with status = "READY" should run the pipeline check to make sure the free inputs exist prior to running
  - if not valid keep status = "STAGED" and return error response


# pipeline log handling
-----

This feature introduces a universal pattern for handling pipeline logs.  Currently, when using kedro-graphql, clients can stream logs during a pipeline's execution, however, after execution there is no way to retrieve the logs.  [kedro hooks](https://docs.kedro.org/en/latest/hooks/introduction.html) are the suggested mechanism for capturing logs in this implemenation, for example,

```python

def save_log_files(catalog):
    ## add "log" text.TextDataset to catalog and then save to preferred storage backend e.g. S3
    ## name of log dataset shouldn't conflict with any existing dataset names but follow a naming convention so it discoverable by clients e.g. 
    ##
    ## <insert uuid here>_logs:
    ##   type: text.TextDataset
    ##
    ## make sure to run kedro_graphql.client.KedroGraphqlClient.updatePipeline mutation to make sure the changes to the data catalog are tracked

@hook_impl
def after_pipeline_run(self, catalog):
    save_log_files(catalog)

@hook_impl
def on_pipeline_error(self, catalog):
    save_log_files(catalog)
```

# client module
-----

The client module is intended to be a low level client for interacting with a remote kedro-graphql api deployment.  It will be leveraged by the [KedroGraphqlRunner](kedrographqlrunner-module#), [viz](viz-module#), and [pipeline log handling](#pipeline-log-handling) features that are part this roadmap.

**organization**

```
src/
  kedro_graphql/
    client.py
```

**implementation**

Here is an example implementation [kedro_graphql_viz_client.ipynb](./kedro_graphql_viz_client.ipynb).

# KedroGraphqlRunner module
-----

The KedroGraphqlRunner is an implementation of kedro's AbstractRunner class.  It allows for easier integration of kedro-graphql into kedro projects.

**organization**

```
src/
  kedro_graphql/
    runners/
      kedro_graphql/
        kedro_graphql.py
```

**usage**

Use just like any kedro runner via cli args or the code api.

```bash
kedro run --runner=kedro_graphql.runners.KedroGraphqlRunner
```

```python

from kedro_graphql.runners import KedroGraphqlRunner

```

**implementation**

`src/kedro_graphl/runners/kedro_graphql/kedro_graphql.py`

```python
from kedro.io import AbstractDataset, DataCatalog, MemoryDataset
from kedro.pipeline import Pipeline
from kedro.runner.runner import AbstractRunner
from pluggy import PluginManager
from kedro_graphql.client import KedroGraphqlClient
from kedro_graphql.models import PipelineInput


class KedroGraphqlRunner(AbstractRunner):
    """``KedroGraphqlRunner`` is an ``AbstractRunner`` implementation. It can be used to 
    run a pipeline on a remote kedro-graphql api.
    """

    def create_default_dataset(self, ds_name: str) -> AbstractDataset:
        """Factory method for creating the default dataset for the runner.

        Args:
            ds_name: Name of the missing dataset
        Returns:
            An instance of an implementation of AbstractDataset to be used
            for all unregistered datasets.

        """
        return MemoryDataset()

    def _run(
        self,
        pipeline: Pipeline,
        catalog: DataCatalog,
        hook_manager: PluginManager = None,
        session_id: str = None,
    ) -> None:
        """The method implementing kedro-graphql pipeline running.

        Args:
            pipeline: The ``Pipeline`` to run.
            catalog: The ``DataCatalog`` from which to fetch data.
            session_id: The id of the session.

        """
        pass
        ## instantiate client, need to figure out how to pass configuration to runner
        # client = KedroGraphqlClient()
        
        ## create PipelineInput object with information passed to this "_run" function
        # pipeline_input = PipelineInput(...)

        ## create pipeline with client
        # pipeline = await client.createPipeline(pipeline_input)
        # self._logger.info("created pipeline with id: " + str(pipeline.id)) 
        
        ## stream pipeline logs
        # self._logger.info("streaming logs")
        # async for e in client.pipelineLogs(pipeline.id):
        #    print(e)
           
        ## read/check result for error or success
        # pipeline = await client.readPipeline(pipeline.id)
        # if pipeline.status == "ERROR":
        #     throw exception
```

# viz module

This section outlines the implementation of a kedro_graphql module named `viz`.  It is intended to serve as an extensible graphical user interface for kedro-graphql projects. The goal of this module is to reduce the complexity of building a graphical interface for a pipeline by providing an opinionated but exensible approach using established component libraries such as [Panel](https://panel.holoviz.org/gallery/index.html) and [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/) in combination with python decorators.  These component libraries are feature rich, allow fast iteration and prototyping.  Robust integrations with the jupyter ecosystem provide flexible options for accessbility, development, and deployment.

**organization**

```
src/
  kedro_graphql/
    viz/
      app.py
      components/
        pipeline_data_explorer.py
        pipeline_form.py
        pipeline_form_browser.py
        pipeline_monitor.py
        pipeline_browser.py
      decorators.py
```

**implementation**

### app
-----

`src/kedro_graphql/viz/app.py`

- Shoud be a servable panel app https://panel.holoviz.org/gallery/index.html
- should be able to start using same mechanism as starting the API server by specifying a `--viz` flag e.g. `kedro gql --viz`
- the following should be configurable
  - base url - e.g. serve at localhost:5001/viz or localhost:5001/my-viz
  - port - port to serve the ui
  - api url - url to kedro_graphql api server e.g. localhost:5000/graphql
  - should support the current `KEDRO_GRAPHQL_IMPORTS` config

Example:
To play with this example, open the following notebook and render it with the panel jupyter extention

[kedro-graphql-viz.ipynb](./kedro-graphql-viz.ipynb)

Here is a screen shot:

![kedro-graphql-viz-ss](./kedro-graphql-viz.png)



### components
-----

`src/kedro_graphq/viz/components/`

The core components of the graphical user interface considered common to all instances are:

- [*pipeline_data_explorer*](#pipeline-data-explorer) - renders provided @gql_data_explorer plugin components
- [*pipeline form browser*](#pipeline-form-browser) - browse all available @gql_form plugin components registered in the project
- [*pipeline form*](#pipeline-form-browser) - renders provided @gql_form plugin components for a specific pipeline
- [*pipeline monitor*](#pipeline-monitor)- monitor a pipelines progress and logs during execution
- [*pipeline browser*](#pipeline-browser) - browse all pipelines with search/filter support 

#### pipeline data explorer
-----

`src/kedro_graphql/viz/components/pipeline_dashboards.py`

- Provided a `kedro_graphql.models.Pipeline` object and client the PipelineDataExplorer component will render the corresponding @gql_data_explorer plugin components.
- If more than one @gql_data_explorer component matches to a pipeline name the component should render the components as seperate tabs or a selection menu.
- @gql_data_explorer plugin components are discovered when the viz app starts up see [extend](#extend) for more information

Usage:

```python
from kedro_graphql.models import Pipeline
from kedro_graphql.viz.components import PipelineDataExplorer
from kedro_graphql.viz.client import KedroGraphqlClient

client = KedroGraphqlClient()

PipelineDataExplorer(client = client, pipeline: Pipeline)

```

Implementation:

- This implementation follows the [Reusable Components](https://panel.holoviz.org/tutorials/intermediate/reusable_components.html#creating-reusable-viewer-components) pattern of the Panel project.


```python
import param
from kedro_graphql.client import KedroGraphqlClient
from kedro_graphql.models import Pipeline
from kedro_graphql.config import config
import panel as pn

class PipelineDataExplorer(pn.viewable.Viewer):
    client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)
    pipeline = param.ClassSelector(class_=Pipeline)
    config = param.Dict()

    def _get_gql_data_explorer(self):
        """
        A factory method for returning the @gql_data_explorer plugins associated with the provided pipline.
        
        Returns:
            @gql_data_explorer plugin (panel.viewable.Viewer) 
        """
        ## more logic need to handle cases such as:
        ## "If more than one @gql_data_explorers component matches to a pipeline name the component should render the components as seperate tabs or a selection menu"
        return config.VIZ_PLUGINS["DATA_EXPLORERS"][self.param.pipeline.name](client = self.param.client, pipeline = self.param.pipeline) 

    def __panel__(self):
        

        return self._get_gql_data_explorer()

```



#### pipeline form browser
-----

`src/kedro_graphql/viz/components/pipeline_form_browser.py`

For pipelines in the project that have a registered @gql_form plugin render them as actionable cards.  The card has two buttons "Run" and "Explore".

**Buttons**

- "Explore" renders a kedro-viz representation of the pipeline using one the methods described here https://docs.kedro.org/projects/kedro-viz/en/stable/kedro-viz_visualisation.html#running-kedro-viz-in-a-notebook
- "Run" - rpasses the name of the pipeline to the PipelineForm component as a parameter to render @gql_form plugins (see [extend](#extend) for information on how the @gql_form decorator is implemented) associated with that pipeline

Implementation:

- This implementation follows the [Reusable Components](https://panel.holoviz.org/tutorials/intermediate/reusable_components.html#creating-reusable-viewer-components) pattern of the Panel project.

In [70]:
## some code commented out because it has not been implemented
import param
#from kedro_graphql.client import KedroGraphqlClient
from kedro_graphql.models import Pipeline
from kedro_graphql.config import config
import panel as pn

class PipelineFormBrowser(pn.viewable.Viewer):
#    client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)
    config = param.Dict()

    def _get_gql_forms(self):
        """
        Return allthe @gql_form plugins registered in the project.
        
        Returns:
            [@gql_form plugin (panel.viewable.Viewer)] 
        """
        ## more logic need to handle cases such as:
        ## "If more than one @gql_form component matches to a pipeline name the component should render the components as seperate tabs or a selection menu"
        return config.VIZ_PLUGINS["FORMS"]

    def __panel__(self):
        #forms =  self._get_gql_forms()
        
        ## some logic here to build panel component from info in "forms" variable
        ## additional calls the API with client can fetch more info such as pipeline description etc...
        ## just an example
        return pn.Row(
            pn.Card(
              pn.Row(
                pn.Card("An example pipeline.", 
                        pn.Row(
                          pn.widgets.Button(name='Run', button_type='success'), 
                          pn.widgets.Button(name='Explore', button_type='primary'),
                        ),
                        sizing_mode = "stretch_width",
                        title='example00'),
                pn.Card("Another example pipeline.",
                        pn.Row(
                          pn.widgets.Button(name='Run', button_type='success'), 
                          pn.widgets.Button(name='Explore', button_type='primary'),
                        ),
                        sizing_mode = "stretch_width",
                        title="example01"),
              ),
              title="Select a pipeline."
            )
        )

PipelineFormBrowser()

PipelineFormBrowser(config=None, name='PipelineFormBrowser01738')

#### pipeline form
-----

`src/kedro_graphql/viz/components/pipeline_form.py`

- Provided the name of the pipeline the PipelineForm component will render the corresponding @gql_form plugin component.
- If more than one @gql_form component matches to a pipeline name the component should render as seperate tabs or a selection menu.
- clicking run should render the PipelineMonitor

Example usage:

```python
from kedro_graphql.viz.components import PipelineForm

from kedro_graphql.viz.client import KedroGraphqlClient

client = KedroGraphqlClient()

PipelineForm(client = client, pipeline: str)

```

Implementation:

- This implementation follows the [Reusable Components](https://panel.holoviz.org/tutorials/intermediate/reusable_components.html#creating-reusable-viewer-components) pattern of the Panel project.


In [71]:
## some code commented out because it has not been implemented
import panel as pn
import param
#from kedro_graphql.client import KedroGraphqlClient
pn.extension()

class PipelineForm(pn.viewable.Viewer):
    #client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)
    pipeline = param.String()

    def _get_gql_forms(self):
        """
        Factory method to return the @gql_form plugins registered to the pipeline.
        
        Returns:
            [@gql_form plugin (panel.viewable.Viewer)]
        """

        return config.VIZ_PLUGINS["FORMS"][self.param.pipeline]

    def __panel__(self):

        #forms = self._get_gql_forms()
        
        ## logic to build form here
        ## more logic need to handle cases such as:
        ## "If more than one @gql_form component matches to a pipeline name the component should render the components as seperate tabs or a selection menu"
        
        return pn.Card("An example pipeline form.", 
            pn.Row(
              pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...'),
            ),
            pn.Row(
              pn.widgets.Button(name='Run', button_type='success'), 
            ),
            sizing_mode = "stretch_width",
            title='Form') # return panel component

PipelineForm()

PipelineForm(name='PipelineForm01787', pipeline='')

#### pipeline monitor
-----

`src/kedro_graphql/viz/components/pipeline_monitor.py`

A component that displays a pipelines progress and logs during execution.


**Usage**

```python
from kedro_graphql.viz.components import PipelineMonitor
from kedro_graphql.client import KedroGraphqlClient
from kedro_graphql.models import Pipeline

client = KedroGraphqlClient()

PipelineMonitor(client = client, pipeline: Pipeline)

```

Implementation:

- This implementation follows the [Reusable Components](https://panel.holoviz.org/tutorials/intermediate/reusable_components.html#creating-reusable-viewer-components) pattern of the Panel project.

In [72]:
import panel as pn
pn.extension("terminal")
from kedro_graphql.models import Pipeline
#from kedro_graphql.client import KedroGraphqlClient

class PipelineMonitor(pn.viewable.Viewer):
    #client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)
    pipeline = param.ClassSelector(class_=Pipeline)

    def __panel__(self):

        ## logic to build form here
        ## display info about current pipeline
        ## subscribe to logs and print to terminal
        ## subscribe to pipeline events

        ## an example
        
        terminal =  pn.widgets.Terminal(
            "Welcome to the Panel Terminal!\nI'm based on xterm.js\n\n",
            options={"cursorBlink": True},
            height=300, sizing_mode='stretch_width'
        )
    
        terminal.write("kedro log messages here")
        
        info = pn.Card("Information about the example pipeline here.", 
                   pn.Row(
                     pn.widgets.Button(name='Explore', button_type='primary'),
                   ),
                   sizing_mode = "stretch_width",
                title='example00')
        
        return pn.Card(
            info,
            terminal,
            title="Monitor"
        )

PipelineMonitor()

PipelineMonitor(name='PipelineMonitor01811', pipeline=None)

#### pipeline browser
-----

`src/kedro_graphql/viz/components/pipeline_browser.py`

A datatable style component with filter and pagination capabilities.  The panel [Tabulator](https://panel.holoviz.org/reference/widgets/Tabulator.html) component is an ideal candidate.  Perhaps in combination with https://panel.holoviz.org/reference/widgets/TextInput.html.

Filter by:

- pipeline attributes e.g. name, id, time, tags, datasets etc.
- calls the readPipelines query on the api

**Buttons**

- button rendered on each row of the datatable name "Open", which will render the PipelineDataExplorer.
- button rendered on each row of the datatable name "Monitor" for pipelines still running that allows user to navigate to PipelineMonitor

**Usage**

```python
from kedro_graphql.viz.components import PipelineBrowser

from kedro_graphql.viz.client import KedroGraphqlClient

client = KedroGraphqlClient()

PipelineBrowser(client = client)

```

Implementation:

- This implementation follows the [Reusable Components](https://panel.holoviz.org/tutorials/intermediate/reusable_components.html#creating-reusable-viewer-components) pattern of the Panel project.

In [73]:
## some code commented out because it has not been implemented
import panel as pn
import param
import pandas as pd
import datetime as dt
import numpy as np
#from kedro_graphql.client import KedroGraphqlClient
pn.extension('tabulator')

class PipelineBrowser(pn.viewable.Viewer):
    #client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)

    def __panel__(self):

        #forms = self._get_gql_forms()
        
        ## logic to build form here

        ## an example
        df = pd.DataFrame({
            'int': [1, 2, 3],
            'float': [3.14, 6.28, 9.42],
            'str': ['A', 'B', 'C'],
            'bool': [True, False, True],
            'date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10)],
            'datetime': [dt.datetime(2019, 1, 1, 10), dt.datetime(2020, 1, 1, 12), dt.datetime(2020, 1, 10, 13)]
        }, index=[1, 2, 3])
        
        
        df_widget = pn.widgets.Tabulator(df, buttons={'Open': "<i class='fa fa-folder-open'></i>"})
        
        return pn.Row(
            pn.Card("Search for a pipeline", 
                    pn.Row(
                      pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...'),
                    ),
                    pn.Row(
                      df_widget
                    ),
                    pn.Row(
                      pn.widgets.Button(name='Open', button_type='success'), 
                    ),
                    sizing_mode = "stretch_width",
                    title='Search'),
        )
        
PipelineBrowser()

PipelineBrowser(name='PipelineBrowser01843')

### extend
-----

Kedro-graphql already uses python decorators that enable one to define custom queries, mutations, and subscriptions ([source](https://github.com/cellsignal/kedro-graphql/blob/main/src/kedro_graphql/decorators.py), [example](https://github.com/cellsignal/kedro-graphql?tab=readme-ov-file#extensible-api)).  Kedro-graphql aims to support two additional decorators to make the graphical interface extensible:

- `@gql_form` - one can define one or more forms for a pipeline
- `@gql_data_explorer` - one can define one or more data explorer components to interpret a pipeline's results


#### decorator implementation
-----

`src/kedro_graphql/viz/decorators.py`

```python
import os
from importlib import import_module
from kedro_graphql.logs.logger import logger

VIZ_PLUGINS = {"FORMS":{},
               "DATA_EXPLORERS":{}
              }

def discover_plugins(config):
    ## call this when starting app to discover plugins
    ## discover plugins e.g. decorated functions e.g @gql_form, etc...
    imports = [i.strip() for i in config["KEDRO_GRAPHQL_IMPORTS"].split(",") if len(i.strip()) > 0]
    for i in imports:
        import_module(i)

def gql_form(pipeline):
    """
    Args:
        pipeline (str): name of pipeline.
    """
    def register_plugin(plugin_class):
        if VIZ_PLUGINS["FORMS"].get(pipeline, False):
            VIZ_PLUGINS["FORMS"][pipeline].append(plugin_class)
        else:
            VIZ_PLUGINS["FORMS"][pipeline] = [plugin_class]
        logger.info("registered type plugin 'form': " + str(plugin_class))
        return plugin_class

    return register_plugin

def gql_data_explorer(pipeline):
    """
    Args:
        pipeline (str): name of pipeline
    """
    def register_plugin(plugin_class):
        if VIZ_PLUGINS["DATA_EXPLORERS"].get(pipeline, False):
            VIZ_PLUGINS["DATA_EXPLORERS"][pipeline].append(plugin_class)
        else:
            VIZ_PLUGINS["DATA_EXPLORERS"][pipeline] = [plugin_class]
        logger.info("registered type plugin 'data_explorer': " + str(plugin_class))
        return plugin_class

    return register_plugin
```

#### decorator usage
-----

- implementation of @gql_form and/or @gql_data_explorer components should follow the [Reusable Components](https://panel.holoviz.org/tutorials/intermediate/reusable_components.html#creating-reusable-viewer-components) pattern of the Panel project.

`@gql_form`

```python
import panel as pn
import param
from kedro_graphql.client import KedroGraphqlClient
from kedro_graphql.viz.decorators import gql_form

@gql_form("example00") ## will be registered to pipeline named "example00"
class MyForm(pn.viewable.Viewer):
    client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)
    pipeline = param.String()

    def __panel__(self):
        
        ## logic to build form here
        
        return pn.Card("An example pipeline form.", 
            pn.Row(
              pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...'),
            ),
            pn.Row(
              pn.widgets.Button(name='Run', button_type='success'), 
            ),
            sizing_mode = "stretch_width",
            title='Form') # return panel component


```

`@gql_data_explorer`
```python
import pandas as pd
import panel as pn
import param
from kedro_graphql.client import KedroGraphqlClient
from kedro_graphql.models import Pipeline
from kedro_graphql.viz.decorators import gql_data_explorer

pn.extension("tabulator")

@gql_data_explorer("example00") ## will be registered to pipeline named "example00"
class DataExplorer(pn.viewable.Viewer):
    client = param.ClassSelector(default=KedroGraphqlClient(), class_=KedroGraphqlClient)
    pipeline = param.ClassSelector(class_=Pipeline)

    def __panel__(self):
        ## data for the component could be retrieve 2 ways:
        ##   - fetch signed Urls via client 
        ##   - or through kedro dataCatalog api e.g. DataCatalog.from_config(self.param.pipeline.data_catalog), this preferred if the ui server is deployed with permissions to read from remote data stores
        data_url = "https://assets.holoviz.org/panel/tutorials/turbines.csv.gz"
        df = pn.cache(pd.read_csv)(data_url)
        return pn.Column(
            pn.widgets.IntSlider.from_param(self.param.page_size, start=5, end=25, step=5),
            pn.widgets.Tabulator.from_param(self.param.data, page_size=self.param.page_size, sizing_mode='stretch_width')
        ) # return panel component
```